# 深入理解Python3中的`super`

## 改变描述器的搜索途径

`super`是一个类型，其对应的C语言结构体为`PySuper_Type`。其主要目的是调用父类中定义的方法。创建`super`对象需要两个参数：`type_`和`obj`，其中`type_`是一个类型对象，`obj`对象是`type_`类型的一个实例。当访问`super`对象的属性时，将在`type_.__mro__`中位于`type(obj)`之后的类型对象中搜索描述器（Descriptor）。其实现代码在`super_getattro()`中。下面通过一个例子解释`super`的属性搜索行为：

In [1]:
class A:
    def func(self):
        print("A.func", self.name)
        
class B(A):
    def __init__(self, name):
        self.name = name
        
    def func(self):
        print("B.func", self.name)

`b.func`将首先在`b`的类型`B`的`__dict__`中搜索`"func"`，由于`B.__dict__["func"]`是一个函数对象，函数对象具有`__get__`属性，因此`b.func`得到的是一个`bound method`对象，该对象对`b`和`B.__dict__["func"]`包装，调用它时相当与调用`B.__dict__["func"](b)`。这就是Python实现方法调用的方式。

In [2]:
b = B("b")
print(b.func)
b.func()

<bound method B.func of <__main__.B object at 0x0000022334F96400>>
B.func b


下面来看`super`是如何改变属性的搜索路径的。`B.__mro__`中有三个元素，其中`A`和`object`位于`B`之后，因此获取`super`对象`sb`的`func`属性时，将依次检查`A`和`object`的`__dict__`中是否有名为`func`的描述器，由于在`A`中找到了，因此将得到调用它的`__get__()`所得到的`bound method`对象。

In [3]:
sb = super(B, b)
print(B.__mro__)

sb.func()

(<class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
A.func b


下面分别查看`b.func`和`sb.func`这两个`bound method`对象中`__func__`属性的函数全名：

In [4]:
print(b.func.__func__.__qualname__, sb.func.__func__.__qualname__)

B.func A.func


## Python3中的super

`super`通常用于调用父类的方法，然而在Python 2中由于使用`super`使需要传递类对象和实例对象，使得调用`super`的语句不简洁，并且一旦需要修改类名，所有的`super`调用都需要修改。在Python 3中这一点得到了改进，例如下面的程序中在`B.func`中使用`super`时，不需要传递任何参数。

In [5]:
class A:
    def func(self):
        print("A.func", self.name)
        
class B(A):
    def __init__(self, name):
        self.name = name
        
    def func(self):
        super().func()
        print("B.func", self.name)
        
b = B("b")
b.func()

A.func b
B.func b


下面我们分析`super()`是如何得到`B`和`self`这两个对象的。

## 获取`self`参数

在类中定义的函数，其第一个参数一定是实例对象，因此只需要找到第一个参数所指向的对象即可。Python中每层函数调用都会创建对应的`frame`（与`PyFrameObject`结构体对应）对象。其最后一个字段`f_localsplus`是可变长的数组：

```c
PyObject *f_localsplus[1];  /* locals+stack, dynamically sized */
```

这个数组中的头一个元素就是传递给与其对应的函数的第一个参数。因此可以在`super`对象的初始化函数`super_init()`中找到这样的语句：

```c
if (type == NULL) {
    /* Call super(), without args -- fill in from __class__
       and first local variable on the stack. */
    PyFrameObject *f;
    PyCodeObject *co;
    Py_ssize_t i, n;
    f = PyThreadState_GET()->frame;
    /* ... */
    obj = f->f_localsplus[0];
}
```

即若没有传递类型对象，则把当前的`frame`对象的`f_localsplus`数组中的第一个元素作为实例对象。

下面通过程序验证上述事实。`get_frame_localsplus(frame, idx)`用于获取`frame`中`f_localsplus`字段的第`idx`元素。关于其具体实现，请读者参考`frame`相关的章节。

In [6]:
import ctypes
import inspect

def get_frame_localsplus(frame, idx):
    code = frame.f_code
    extras = code.co_nlocals + len(code.co_cellvars) + len(code.co_freevars) + code.co_stacksize
    addr = id(frame) + frame.__sizeof__() - (extras - idx) * ctypes.sizeof(ctypes.c_ssize_t)
    return ctypes.cast(addr, ctypes.POINTER(ctypes.c_ssize_t))[0]

class A:
    def first_locals_is_self(self):
        frame = inspect.currentframe()
        first_ref = get_frame_localsplus(frame, 0)
        return first_ref

在`A.first_locals_is_self`中，首先通过`inspect.currentframe()`获得当前运行中的`frame`对象，然后调用`get_frame_localsplus()`获得其中`f_localsplus`字段的第0个元素。下面验证这第0个元素就是实例`a`的地址：

In [7]:
a = A()
print(id(a), a.first_locals_is_self())

2350235907800 2350235907800


## 获取当前的类

为了在`super_init()`中获取包含正在被调用的函数的类，Python 3对编译器进行了一些改动。为了理解这些改动，我们先看看类是如何被创建的。当Python编译器遇到`class`关键字定义类时，将在一个新的运行环境（字典）中运行`class`中的代码。然后调用`type()`创建类型对象。下面是与`class`关键字等效的代码。

In [8]:
env = {}

exec("""
def func(self):
    print(self)
""",  env)

A = type("A", (object,), env)

a = A()
a.func()

由上述的过程可知，当类中的方法函数定义时，类型对象还未被创建，因此Python 3在类型对象创建之后，对其中包含`super`的函数进行处理。为这些函数创建`__closure__`属性，在其中使用`__class__`引用该函数所属的类。

下面看一个例子：

In [9]:
class A:
    
    def func_with_super(self):
        super
        
    def func_no_super(self):
        pass

`A.func_with_super()`中引用了`super`符号，因此Python编译器会在创建类型`A`时，为该函数添加一个名为`__class__`的`freevar`变量，该变量引用`__closure__[0].cell_contents`，即类型对象`A`：

In [10]:
print(A.func_with_super.__code__.co_freevars)
print(A.func_with_super.__closure__[0].cell_contents)

('__class__',)
<class '__main__.A'>


在`A.func_no_super()`中没有引用`super`符号，因此其`__closure__`为空：

In [11]:
A.func_no_super.__closure__

注意Python编译器只检查函数中是否有`super`的引用，而不关心它实际内容是什么。在下面的例子中`func_with_mysuper`的`__closure__`为空，而`func_with_super`的`__closure__`则有类型`B`的引用。

In [12]:
mysuper, super = super, "something"

class B:
    def func_with_mysuper(self):
        mysuper
        
    def func_with_super(self):
        super
        
    def func_with_super2(self):
        super = 2
        super
        
print(B.func_with_mysuper.__closure__)        
print(B.func_with_super.__closure__)
print(B.func_with_super2.__closure__)        
super = mysuper

None
(<cell at 0x0000022334FA6408: type object at 0x0000022335411AE8>,)
(<cell at 0x0000022334FA6408: type object at 0x0000022335411AE8>,)


相关的代码：

* `symtable.c`中`case Name_kind:`部分检查是否存在`super`的引用。如果存在则添加一个`__class__`引用。
* `compile.c`中`if (u->u_ste->ste_needs_class_closure) {`的部分检查函数是否存在`__class__`引用，如果存在，则产生相应的代码。
* 创建类时会调用内置函数`__build_class__()`执行创建类的代码。
* 在创建类的代码中可以找到创建`__class__`的语句。

In [13]:
def test():
    class A:
        def func(self):
            super
    return A

import dis
dis.dis(test)

  2           0 LOAD_BUILD_CLASS
              1 LOAD_CONST               1 (<code object A at 0x0000022334FA80C0, file "<ipython-input-13-794cc6fd2a02>", line 2>)
              4 LOAD_CONST               2 ('A')
              7 MAKE_FUNCTION            0
             10 LOAD_CONST               2 ('A')
             13 CALL_FUNCTION            2 (2 positional, 0 keyword pair)
             16 STORE_FAST               0 (A)

  5          19 LOAD_FAST                0 (A)
             22 RETURN_VALUE


上面的代码动态地创建了一个函数，并且把它作为参数传递给了`__build_class__()`函数。这个被创建的函数的代码如下：

In [14]:
class_code = test.__code__.co_consts[1]
dis.dis(class_code)

  2           0 LOAD_NAME                0 (__name__)
              3 STORE_NAME               1 (__module__)
              6 LOAD_CONST               0 ('test.<locals>.A')
              9 STORE_NAME               2 (__qualname__)

  3          12 LOAD_CLOSURE             0 (__class__)
             15 BUILD_TUPLE              1
             18 LOAD_CONST               1 (<code object func at 0x0000022334CDAC90, file "<ipython-input-13-794cc6fd2a02>", line 3>)
             21 LOAD_CONST               2 ('test.<locals>.A.func')
             24 MAKE_CLOSURE             0
             27 STORE_NAME               3 (func)
             30 LOAD_CLOSURE             0 (__class__)
             33 RETURN_VALUE


可以看到函数的返回值是一个`cell`对象。在`__build_class__`的代码中，如果`cell`变量通过`PyCell_Check(cell)`检查，则调用`PyCell_Set(cell, cls)`将该`cell`的内容设置为被创建的类：

```c
cell = PyEval_EvalCodeEx(PyFunction_GET_CODE(func), PyFunction_GET_GLOBALS(func), ns,
                         NULL, 0, NULL, 0, NULL, 0, NULL,
                         PyFunction_GET_CLOSURE(func));
if (cell != NULL) {
    PyObject *margs;
    margs = PyTuple_Pack(3, name, bases, ns);
    if (margs != NULL) {
        cls = PyEval_CallObjectWithKeywords(meta, margs, mkw);
        Py_DECREF(margs);
    }
    if (cls != NULL && PyCell_Check(cell))
        PyCell_Set(cell, cls);
    Py_DECREF(cell);
}
```